# ALW Data Analytics

In [ ]:
import pandas as pd
from sklearn import preprocessing as pre
import numpy as np
from functools import reduce

# Reload spreadsheet using new upload
file_path = "C:/Users/blaks/OneDrive/Documents/Work/CCM Analysis/ALW 24_25 All Stats (WyScout).xlsx"
xls = pd.ExcelFile(file_path)

# Reload the Excel file
xls = pd.ExcelFile(file_path)

# Include all relevant performance-related sheets
all_perf_sheets = [
    'Attacking', 'Defending', 'Passing', 'Key Passing'
       ]

# Load and merge all these sheets
dfs = [xls.parse(sheet) for sheet in all_perf_sheets]
merged_df = reduce(lambda left, right: pd.merge(left, right, on='Player', how='outer'), dfs)

# Load general data for position and minutes
general_df = xls.parse('General')[['Player', 'Position', 'Minutes played']]
full_df = pd.merge(merged_df, general_df, on='Player', how='left')
full_df = full_df.fillna(0)
# Remove duplicate players, keeping the one with the most minutes played
full_df = full_df.loc[full_df.groupby('Player')['Minutes played_y'].idxmax()].reset_index(drop=True)

# Drop non-performance identifiers and irrelevant fields
id_cols = ['Player', 'Position_y', 'Minutes played_y']
non_performance_cols = [
    col for col in full_df.columns
    if any(keyword in col.lower() for keyword in ['team', 'age', 'passport', 'height', 'weight', 'birth', 'left', 'right', 'matches', 'penalty'])
    or col.endswith('_x')
    or not any(keyword in col.lower() for keyword in ['90', '%'])
]

stats_df = full_df.drop(columns=non_performance_cols, errors='ignore')



In [64]:

# Keep ID columns separately
player_info = full_df[id_cols]
performance_data = stats_df.drop(columns=id_cols, errors='ignore')

# Convert all columns to numeric and coerce errors
performance_data = performance_data.apply(pd.to_numeric, errors='coerce')

# Group duplicate columns by stripped base names and resolve them
def clean_duplicate_columns(df):
    base_names = {}
    for col in df.columns:
        base = col.strip().replace(" / ", " per ").replace("  ", " ").strip()
        base_names.setdefault(base, []).append(col)

    resolved_df = pd.DataFrame(index=df.index)
    for base, cols in base_names.items():
        if len(cols) == 1:
            resolved_df[base] = df[cols[0]]
        else:
            # Take the column with fewer NaNs or average them
            subset = df[cols]
            if subset.nunique(axis=1).eq(1).all():
                resolved_df[base] = subset.iloc[:, 0]  # identical values
            else:
                resolved_df[base] = subset.mean(axis=1, skipna=True)  # average similar metrics
    return resolved_df

cleaned_perf_data = clean_duplicate_columns(performance_data)

# Standardize using z-scores
scaler = pre.StandardScaler()
standardized_data = pd.DataFrame(scaler.fit_transform(cleaned_perf_data), columns=cleaned_perf_data.columns)

# Reattach player info
final_standardized_df = pd.concat([player_info, standardized_data], axis=1)

# Output list of cleaned, unique standardized stat names
cleaned_perf_data.columns.tolist()

final_standardized_df.to_csv("C:/Users/blaks/OneDrive/Documents/Work/CCM Analysis/ALW 24_25 All Stats (WyScout) - Cleaned.csv", index=False)
final_standardized_df

,Player,Position_y,Minutes played_y,Duels per 90,"Duels won, %",Successful attacking actions per 90,Goals per 90,xG per 90,Head goals per 90,Shots per 90,...,"Accurate smart passes, %_y",Key passes per 90_y,Passes to final third per 90_y,"Accurate passes to final third, %_y",Through passes per 90_y,"Accurate through passes, %_y",Deep completions per 90_y,Deep completed crosses per 90_y,Progressive passes per 90_y,"Accurate progressive passes, %_y"
0,A. Abbott,"RB, RCB, RDMF",67,-0.473532,0.105134,-1.047681,-0.607997,-0.233101,-0.289227,0.299635,...,-0.628595,-0.789914,2.174999,1.795093,-0.875224,-0.777485,-1.093512,-0.781776,0.763161,0.565242
1,A. Apostolakis,RB,1960,-0.159175,0.238629,-0.461842,-0.152437,-0.740798,-0.289227,-0.532984,...,-0.628595,-0.429602,0.079768,1.069334,-0.438359,-0.777485,-0.656257,-0.356245,0.873989,0.079591
2,A. Bennett,"RB, RCB",697,0.008181,0.448565,-0.339792,-0.607997,-0.804260,-0.289227,-0.914184,...,-0.628595,-0.789914,0.525305,0.052421,0.702345,0.072175,-0.777716,-0.474448,1.124983,-0.490225
3,A. Briedis,"CF, LWF, LCMF",81,2.154403,-1.321766,0.984450,5.010574,-0.486949,-0.289227,0.068909,...,-0.628595,-0.789914,0.184128,1.289704,-0.875224,-0.777485,1.602895,-0.781776,-0.778657,0.565242
4,A. Cassar,"LW, CF, RWB",104,-0.034789,-0.725747,-1.047681,-0.607997,-0.804260,-0.289227,-1.044594,...,-0.628595,-0.789914,-0.209230,0.109991,-0.875224,-0.777485,-1.093512,1.274954,-1.300202,-0.306211
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,T. Yallop,"CF, AMF, RWF",1648,-0.281299,-0.084861,1.149217,1.872273,1.543837,3.367869,1.694021,...,0.161750,0.090848,0.922677,0.417660,0.847967,0.203023,0.752677,0.518456,-0.208217,0.902427
256,T. Younis,"CF, LW, LAMF",1478,0.693434,0.005982,0.734248,-0.607997,-0.233101,-0.289227,0.600582,...,-0.628595,0.691367,0.063712,0.318092,1.042129,0.203023,-0.194709,0.376613,-0.354902,1.000707
257,Z. Kruger,"RCMF, LCMF",1062,0.134828,0.080208,-0.376407,-0.203055,-0.677336,-0.289227,-0.282195,...,2.532784,-0.109325,0.408904,0.789978,-0.050034,0.284591,-0.680548,-0.190761,-0.152803,0.199828
258,Z. McMeeken,"RB, RWB",991,1.273524,0.192100,0.447430,-0.152437,-0.106177,1.767890,0.590550,...,2.532784,-0.069291,1.320048,0.817819,0.435372,0.638474,0.218254,1.369517,1.480286,0.919678
